In [ ]:
from torch.utils.data import random_split

from scripts.avi_rnn import *
from scripts.utils import set_seed
from scripts.ds_class import *
from scripts.homeos import *
from scripts.plotting import *
from scripts.fit_motif import *
from scripts.time_series import *
from scripts.ra import *
exp_folder = "experiments\\avi"

In [ ]:
random_seed = 42
np.random.seed(random_seed)
T=25.6/2
dt=.1 
batch_size=50

In [ ]:
task = angularintegration_task_constant(T=T, dt=dt, speed_range=[0.,0.], sparsity=1, random_angle_init='equally_spaced');
for N in [64]:
    sub_exp = f"N{N}_T128_noisy"
    for activation in ["recttanh", "relu", "tanh"]:
        folder = os.path.join(exp_folder, sub_exp, activation)
        exp_list = glob.glob(os.path.join(folder, "res*"))
        nact_exp = exp_folder +  "/all_trajs" + f"/N{N}_{activation}"
        print(f"Processing {nact_exp}")
        for exp_i in range(len(exp_list)):
            path = exp_list[exp_i]
            try:
                net, result = load_net_path(path)
            except:
                print(f"Error loading {path}")
                continue
            net.eval()    
            input, target, mask, output, trajectories = simulate_rnn_with_task(net, task, T, '', batch_size=batch_size)
            net_id = os.path.basename(path).split("_")[-1].split(".")[0]
            print(net_id, trajectories.shape)
            os.makedirs(nact_exp, exist_ok=True)
            np.save(f'{nact_exp}/trajectories_{net_id}.npy', trajectories.copy())


In [ ]:
def run_on_target(target_name, save_dir, data_dir, ds_motif = 'ring', analytic = True,
                    alpha_init = None, velocity_init = None, vf_on_ring_enabled = False, #if analytic then not used
                    homeo_type = 'node', layer_sizes = 1*[64], batch_size = 32,
                    train_ratio = 0.8, training_pairs = False, 
                    lr = 0.01, num_epochs = 1000, 
                    random_seed = 313):
    save_dir = os.path.join(save_dir, ds_motif)
    os.makedirs(save_dir, exist_ok=True)
    set_seed(random_seed)
     
    trajectories_target = np.load(data_dir / f'{target_name}.npy')
    dim = trajectories_target.shape[2]
    tsteps = trajectories_target.shape[1]
    B = trajectories_target.shape[0]
    
    homeo_type = 'node'
    dt = .1
    time_span = torch.tensor([0.0, T-dt/2.])
    if training_pairs:
        time_span = torch.tensor([0.0, dt])
    ds_motif = 'ring'
    ds_params = {
        'ds_motif': ds_motif, 'dim': dim, 'dt': dt, 'time_span': time_span,
        'analytic': analytic, 'vf_on_ring_enabled': vf_on_ring_enabled,
        'alpha_init': alpha_init, 'velocity_init': velocity_init
    }
    homeo_params = {'homeo_type': homeo_type, 'dim': dim, 'layer_sizes': layer_sizes, 'activation': nn.ReLU, 'init_type': None}
    training_params = {'lr': lr,'num_epochs': num_epochs,'annealing_params': {'dynamic': False, 'initial_std': 0.0, 'final_std': 0.0},'early_stopping_patience': 1000,'batch_size': batch_size,'use_inverse_formulation': True}
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair(torch.tensor(trajectories), training_pairs=training_pairs)

    train_ratio = 0.8
    n_train = int(train_ratio * B)
    n_test = B - n_train
    train_set, test_set = random_split(trajectories_target_full, [n_train, n_test])
    trajectories_target_train = trajectories_target[train_set.indices]
    trajectories_target_test = trajectories_target[test_set.indices]

    # target_ra_points = get_homeo_invman(interpolated_homeo)
    target_ra_points = trajectories_target[:, -1, :].reshape(-1, dim)
    #target_ra_points = (target_ra_points - mean.detach().numpy()) / std.detach().numpy()

    homeo = build_homeomorphism(homeo_params)
    source_system_ra = build_ds_motif(**ds_params)
    homeo_ds_net = Homeo_DS_Net(homeo, source_system_ra)
    homeo_ds_net, losses, grad_norms = train_homeo_ds_net_batched(
        homeo_ds_net=homeo_ds_net,
        trajectories_target=trajectories_target_train,
        **training_params
    )
    homeo_ds_net.eval()

    #test
    _, _, training_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_train)
    _, _, test_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_test)
    traj_src_np, traj_trans_np, _ = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target)
    inv_man = homeo_ds_net.invariant_manifold(100).detach().numpy()
    jac_norm_frobenius = jacobian_norm_over_batch(homeo_ds_net.homeo_network, trajectories_target.reshape(-1,dim), norm_type='fro').detach().numpy()
    jac_norm_spectral = jacobian_norm_over_batch(homeo_ds_net.homeo_network, trajectories_target.reshape(-1,dim), norm_type='spectral').detach().numpy()

    np.savez(
    f"{save_dir}/results_{target_name}.npz",
    jac_fro=jac_norm_frobenius,
    jac_spec=jac_norm_spectral,
    training_loss=training_loss,
    test_loss=test_loss,
    losses=np.array(losses),  
    grad_norms=np.array(grad_norms),
    inv_man=inv_man,
    target_ra_points=target_ra_points,
)

    save_homeo_ds_net(homeo_ds_net, f"{save_dir}/homeo_{target_name}.pth")
    np.save(f"{save_dir}/traj_motif_transformed_{target_name}.npy", traj_trans_np) 
    np.save(f"{save_dir}/traj_motif_source_{target_name}.npy", traj_src_np) 

In [ ]:
N = 64
activation = "recttanh"
exp_dir = Path('experiments')
data_dir = exp_dir  / 'avi' / "all_trajs" / f"N{N}_{activation}"

npy_files = list(data_dir.glob('*.npy'))
print(npy_files)
for file in npy_files:
    target_name = file.name.removesuffix('.npy')
    save_dir = exp_dir / 'avi' / "fit_motifs" / f"N{N}_{activation}"
    print(target_name)
    run_on_target(target_name, save_dir, data_dir, ds_motif = 'ring')